#  Main Idea

1) Getting input of puzzle file from user. There are two results from this input:

* The file does not exist. Proceed to get user input of the file. After that output the puzzle file in a .txt file.

* The file exists. This file will be used.

2) Based on the input file, a integer (binary) programming problem is formed. Check ILP Formulation section for more info.

3) Solve the integer programming problem.

4) Based on the results, the output is printed into the console and a separate .txt file.

# Integer (Binary) Linear Programming Formulation

Note that solving a sudoku puzzle requires us to fill in numbers into each empty cell based on the following rules:

1) Each number can only appear once in each row

2) Each number can only appear once in each column

3) Each number can only appear once in each $3 \times 3$ box

4) In other words, each number can only appear exactly 9 times in the entire puzzle

This is the most important aspect in solving the sudoku puzzle. It is based on the following key points:

1) We first want to think of this as a $9 \times 9 \times 9$ array to represent the puzzle. Let's call this 3D array $M$. Another way of thinking about $M$ is that each cell in the $9 \times 9$ sudoku puzzle has a vector of length 9.

2) In any cell, its vector represents the value it takes. For example, if the cell in row 3 column 4 has value 8, then the vector that represents cell (3,4) has value 1 in the 8th entry and 0 for the rest of them.

3) Based on the rules of solving sudoku puzzles above, we can write the following constraints:

* $\sum_{k = 1}^{9} m_{i,j,k} = 1$ for all $i,j = 1, \ldots, 9 \Longrightarrow$ This constraint ensures that each cell $(i,j)$ can contain exactly 1 number.

* $\sum_{j = 1}^{9} m_{i,j,k} = 1$ for all $i,k = 1, \ldots, 9 \Longrightarrow$ This constraint ensures that each value $k$ can only appear exactly once in each row $i$.

* $\sum_{i = 1}^{9} m_{i,j,k} = 1$ for all $j,k = 1, \ldots, 9 \Longrightarrow$ This constraint ensures that each value $k$ can only appear exactly once in each column $j$.

* $\sum_{j = 3y+1}^{3y+3} \sum_{i = 3x+1}^{3x+3} m_{i,j,k} = 1$ for $x,y = 0,1,2$ and $k = 1, \ldots, 9 \Longrightarrow$ This constraint ensures that each value $k$ can only appear exactly once in each $3 \times 3$ box.

4) Now that we have the set of constraints that are based on the rules of the sudoku game, we need to further add a few more constraints to represent the values presented in the puzzle (remember that the puzzle that we start with is not empty and contains numbers on certain cells). Essentially, the constraints are just 

$$m_{i,j,k} = 1 \text{ if pre-filled cell $(i,j)$ has value $k$}$$

5) However, to solve an integer linear programming problem, an objective function is also needed. In this case, the objective function is set as 

$$\min \ \sum_{i,j,k} m_{i,j,k}$$

The result of the objective value is mainly used to make sure that all 81 cells are filled. In essence, we can choose any objective function that we want as it does not really have any impact on the results.

6) The next issue we face is that most programming languages (Python in this case) solves integer linear programs with constraints of the form

$$Ax <= b$$

where $A$ is a 2D matrix and $x$ and $b$ are vectors. This poses as a problem since our 3D array $M$ is suppose to take the place of vector $x$. To solve this problem, we transform $M$ into vector $X$ which takes the form 

$$\begin{bmatrix} m_{1,1,1} & \ldots & m_{1,1,9} & m_{1,2,1} & \ldots & m_{1,2,9} & \ldots & \ldots & m_{9,9,9} \end{bmatrix}^T$$

Essentially, the first 9 entries are used to represent cell (1,1), the next 9 entries are used to represent cell (1,2), and the next 9 entries are used to represent cell (1,3), and so on. Hence, the vector $X$ has a total of $9 \times 9 \times 9 = 729$ entries. From this point onwards, we will rename the elements in vector $X$ into $\begin{bmatrix} x_1 & \ldots & x_{729} \end{bmatrix}^T$ for simplicity.

7) With the change in representation of $M$, our objective function and constraints will be transformed to the following:

* $\min \ \sum_{i,j,k} m_{i,j,k} \Longrightarrow \min \sum_{i=1}^{729} x_i$

* $\sum_{k = 1}^{9} m_{i,j,k} = 1 \Longrightarrow \sum_{k = 1}^{9} x_{81 \cdot i + 9 \cdot j + k} = 1 \text{for } i = 0 \ldots 8,j = 0 \ldots 8$

* $\sum_{j = 1}^{9} m_{i,j,k} = 1 \Longrightarrow \sum_{j = 0}^{8} x_{81 \cdot i + 9 \cdot j + k} = 1 \text{for } k = 1 \ldots 9,i = 0 \ldots 8$

* $\sum_{i = 1}^{9} m_{i,j,k} = 1 \Longrightarrow \sum_{i = 0}^{8} x_{81 \cdot i + 9 \cdot j + k} = 1 \text{for } k = 1 \ldots 9,j = 0 \ldots 8$

* $\sum_{j = 3y+1}^{3y+3} \sum_{i = 3x+1}^{3x+3} m_{i,j,k} = 1 \Longrightarrow \sum_{j = 3y+1}^{3y+3} \sum_{i = 3x+1}^{3x+3} x_{81 \cdot i + 9 \cdot j + k} = 1 \text{ for } x,y = 0,1,2 \text{ and } k = 1, \ldots, 9$

* $m_{i,j,k} = 1 \Longrightarrow x_{81 \cdot (i-1) + 9 \cdot (j-1) + k} = 1 \text{ if pre-filled cell $(i,j)$ has value $k$}$

Notice that the first 4 sets of constraints each have 81 constraints, and the last constraint has strictly less than 81 constraints (if it has 81 constraints then the sudoku puzzle is completely filled). Therefore, we should expect our total number of constraints to be between 324 and 405, exclusive of those two numbers.

# Algorithm

1) start by inputting the sudoku problem

* can be done by reading in .txt file 

* if no .txt file, can key in the values for it to generate .txt file

2) based on .txt file, put in the "additional rows" of constraints, add it to the $324 \times 729$ matrix, call this matrix `constraint`

3) we also need a matrix $b$ such that $Ax=b$, hence $b$ is a vectors of 1's with the same number of rows in `constraint`

4) run the ILP and obtain the solution vector $x$

5) use double for loops to extract the values 

    for i = 0 to 8
        for j = 0 to 8
            new x = x[81i+9j+1 ... 81i+9j+9]
            idx = index of x == 1
            print (idx)
        end
        print newline
    end

# Next Steps: Making an app

### Goal: To make an app that starts out with an empty sudoku grid with 9x9 cells, let user enter the puzzle, select Solve, and obtain the solution, then click Restart option to start again

Resources:

* http://newcoder.io/gui/part-2/

* https://realpython.com/mobile-app-kivy-python/